# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json



# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
cities_df =pd.read_csv("../WeatherPy/output_data/cities.csv")

#drop na value(if there were) and remove unnamed column
cities_df=cities_df.drop(columns=["Unnamed: 0"])
cities_df = cities_df.dropna()

cities_df

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = cities_df [["Lat", "Lng"]].astype(float)
weights = cities_df["Humidity"].astype(float)


# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights , 
                               dissipating=False, max_intensity= max(weights),
                                 point_radius = 5)

fig.add_layer(heat_layer)


# Saves an image of our chart so that we can view it in a folder
plt.savefig("output_data/Fig1.png") 

fig




# #Analysis: It seems that the humidity is higher in South America in comparison to north america humidity is lower in Algeria, Sudan, Egypt

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
## Narrow down the DataFrame to find your ideal weather condition
new_cities = pd.DataFrame(cities_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

# The ideal condition based on the instruction:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
ideal_temp = (new_cities["Max Temp"]<80) & (new_cities["Max Temp"]>70)
ideal_wind = new_cities["Wind Speed"]<10
ideal_cloudiness = new_cities["Cloudiness"]==0

ideal_cities = new_cities.loc[ideal_cloudiness & ideal_temp & ideal_wind]


In [ ]:
ideal_cities

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = pd.DataFrame(cities_df, columns=["City", "Country", "Lat", "Lng"])

#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
hotel_name = []

# params dictionary to update each iteration

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key,
}



# Loop through the hotel_df and run a lat/long search for each city
for index, row in hotel_df.iterrows():   
# get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
# change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

# Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# make request 
    hotel_names = requests.get(base_url, params=params).json()

    try:
        hotel_df.loc[index,"Hotel Name"]=hotel_names["results"][0]["name"]

    except IndexError:
        print("Missing field/result... skipping.")
        
# Save Data to csv
hotel_df.to_csv("output_data/hotel.csv")

# Visualize to confirm airport data appears
hotel_df




Export data to CSV

In [ ]:
hotel_df.to_csv("output_data/hotel.csv")

#pprint the hotel names


In [ ]:
print(json.dumps(hotel_names, indent=4, sort_keys=True))


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

Add marker layer ontop of heat map


In [ ]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

Add the layer to the map


In [ ]:
fig.add_layer(markers)


In [ ]:
# Display Map
fig